In [1]:
import pandas as pd
import numpy as np
import random 
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, GlobalMaxPooling1D, concatenate, GlobalAveragePooling1D

In [2]:
db = pd.read_csv('NIFTY50_all.csv', header=0) #dataset

In [3]:
print(db.head())

x = {} #company set

for i in range(len(db)):
    x[db.Symbol[i]] = 0

         Date      Symbol Series  Prev Close    Open     High    Low   Last  \
0  2007-11-27  MUNDRAPORT     EQ      440.00  770.00  1050.00  770.0  959.0   
1  2007-11-28  MUNDRAPORT     EQ      962.90  984.00   990.00  874.0  885.0   
2  2007-11-29  MUNDRAPORT     EQ      893.90  909.00   914.75  841.0  887.0   
3  2007-11-30  MUNDRAPORT     EQ      884.20  890.00   958.00  890.0  929.0   
4  2007-12-03  MUNDRAPORT     EQ      921.55  939.75   995.00  922.0  980.0   

    Close    VWAP    Volume      Turnover  Trades  Deliverable Volume  \
0  962.90  984.72  27294366  2.687719e+15     NaN           9859619.0   
1  893.90  941.38   4581338  4.312765e+14     NaN           1453278.0   
2  884.20  888.09   5124121  4.550658e+14     NaN           1069678.0   
3  921.55  929.17   4609762  4.283257e+14     NaN           1260913.0   
4  969.30  965.65   2977470  2.875200e+14     NaN            816123.0   

   %Deliverble  
0       0.3612  
1       0.3172  
2       0.2088  
3       0.2735  
4

In [4]:
min_return = min(db[['Close']].min(axis=0))
max_return = max(db[['Close']].max(axis=0))

db['Close'] = (db['Close'] - min_return) / (max_return - min_return) #normalisation (stock prices are now between 0-1)

#db['Close'] = (db['Close'] * (max_return - min_return)) + min_return

In [5]:
print(len(x))

#print(db.Date[:20])
#print(db.Symbol[:20])

sym = db.Symbol[0]

temp = [[]]*65      #company wise matrix
#print(temp)
count = 0           #company number

for i in range(len(db)):
    if db.Symbol[i] == sym:
        temp[count].append(db.Close[i])
    else:
        count += 1
        sym = db.Symbol[i]

65


In [6]:
N = 10              #number of days
n_rows = 10000      #10K rows ~ 11 mins of training
X = []              #input
Y = []              #output

In [7]:
for i in range(65):                         #company
    for k in range(n_rows//65):             #fixed number of datapoints       
        j = random.randint(0, len(temp[i])-N)           #choose starting point
        X.append(temp[i][j:j+N])            #add 10 days from starting point to input
        Y.append(temp[i][j+N])              #11th day is output

In [8]:
len(X)

9945

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [10]:
def create_model():
    in_seq = Input(shape = (N, 1))

    x = Bidirectional(LSTM(128, return_sequences=True))(in_seq)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x) 
            
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    out = Dense(1, activation="linear")(conc)      

    model = Model(inputs=in_seq, outputs=out)
    model.compile(loss="mse", optimizer="adam", metrics=['mae', 'mape'])    
    return model

In [11]:
model = create_model()
model.fit(X_train, y_train, batch_size=2048, epochs=200)

Epoch 1/200
4/4 [==============================] - 15s 857ms/step - loss: 0.0068 - mae: 0.0349 - mape: 170.3420
Epoch 2/200
4/4 [==============================] - 3s 774ms/step - loss: 0.0013 - mae: 0.0272 - mape: 414.1307
Epoch 3/200
4/4 [==============================] - 3s 763ms/step - loss: 9.5648e-04 - mae: 0.0251 - mape: 282.2142
Epoch 4/200
4/4 [==============================] - 3s 725ms/step - loss: 3.8800e-04 - mae: 0.0152 - mape: 252.9227
Epoch 5/200
4/4 [==============================] - 3s 729ms/step - loss: 2.3957e-04 - mae: 0.0088 - mape: 145.6939
Epoch 6/200
4/4 [==============================] - 3s 808ms/step - loss: 1.6626e-04 - mae: 0.0091 - mape: 66.7686
Epoch 7/200
4/4 [==============================] - 3s 789ms/step - loss: 5.8889e-05 - mae: 0.0049 - mape: 58.5237
Epoch 8/200
4/4 [==============================] - 3s 776ms/step - loss: 1.0244e-04 - mae: 0.0064 - mape: 107.7985
Epoch 9/200
4/4 [==============================] - 3s 704ms/step - loss: 5.9152e-05 - mae

In [12]:
results = model.evaluate(X_test, y_test, batch_size=2048)

2/2 [==============================] - 2s 197ms/step - loss: 2.3680e-05 - mae: 0.0025 - mape: 43.4198


In [13]:
results = model.predict(X_test[1:2])

print(X_test[1], results, y_test[1])

[0.028003701358788297, 0.02693834315492135, 0.02738274971996299, 0.02734926703355574, 0.026197158233088203, 0.025314432864169876, 0.02507701017873667, 0.025632518385038723, 0.02576492719037647, 0.026369137485998156] [[0.02768084]] 0.026758754200555207


In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 1)]      0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 10, 256)      133120      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 10, 256)      394240      bidirectional[0][0]              
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 10, 128)      164352      bidirectional_1[0][0]            
______________________________________________________________________________________________